# 캐글 노트북

In [ ]:
%%bash
mkdir -p ~/.kaggle
cat <<EOF > ~/.kaggle/kaggle.json
{"username":"leeyounghoeddie","key":"379cf3724c7475897d95fa4675cd5bdd"}
EOF
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ai03-level1-project

 99% 3.88G/3.91G [00:09<00:00, 500MB/s]
100% 3.91G/3.91G [00:09<00:00, 435MB/s]


In [ ]:
!unzip -q ai03-level1-project.zip -d ./data

# 환경설정

In [ ]:
import torch
print(torch.__version__, torch.version.cuda)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -o "/content/drive/MyDrive/(개인)AI_3기/프로젝트/초급프로젝트/pill_images_v3.zip" -d /content/

2.7.1+cu118 11.8 이어야 함.

In [ ]:

# 1) 기존 설치된 torch/torchvision/torchaudio 제거
!pip uninstall -y torch torchvision torchaudio

# 2) Colab (2.0.1+cu118) 다시 설치
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 \
  --extra-index-url https://download.pytorch.org/whl/cu118


런타임 → 런타임 다시 시작

In [ ]:
import torch
print(torch.__version__, torch.version.cuda)

In [ ]:
!pip install --upgrade "torchvision>=0.16.0" \
  --extra-index-url https://download.pytorch.org/whl/cu118

런타임 → 런타임 다시 시작

In [ ]:
import torch
print(torch.__version__, torch.version.cuda)

2.7.1+cu118 11.8 이어야 함.

한글 세팅

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

In [3]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import pandas as pd

from pathlib import Path

import torchvision
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
import torchvision.transforms.v2 as v2
from torchvision.transforms import v2 as T
from PIL import Image, ImageEnhance, ImageFilter
from torch.utils.data import Dataset, DataLoader

In [ ]:
print("PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
device = torch.device("cuda" if torch.backends.mps.is_available() else "cpu")
print(device)

os.makedirs("pth", exist_ok=True)

In [4]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## 한글세팅

※ macOS에서 구동하는 한글 세팅임.

In [ ]:
# macOS용 한글 폰트 설정
"""
# macOS에서 사용 가능한 한글 폰트들
macos_korean_fonts = [
    '/System/Library/Fonts/AppleGothic.ttf',
    '/System/Library/Fonts/AppleSDGothicNeo.ttc',
    '/Library/Fonts/NanumGothic.ttf',
    '/Library/Fonts/NanumBarunGothic.ttf',
    '/System/Library/Fonts/PingFang.ttc'
]

# 사용 가능한 폰트 찾기
available_font = None
for font_path in macos_korean_fonts:
    if os.path.exists(font_path):
        available_font = font_path
        print(f"사용 가능한 폰트 발견: {font_path}")
        break

if available_font:
    # 폰트 설정
    font_prop = fm.FontProperties(fname=available_font)
    plt.rcParams['font.family'] = font_prop.get_name()
    plt.rcParams['axes.unicode_minus'] = False
    print(f"폰트 설정 완료: {font_prop.get_name()}")
else:
    # 기본 폰트로 설정
    plt.rcParams['font.family'] = 'AppleGothic'
    plt.rcParams['axes.unicode_minus'] = False
    print("기본 AppleGothic 폰트 사용")
"""

## 함수화

In [5]:
# 모델 저장 함수
def save_model(model, save_path, model_name="drug_detection_model"):
    """훈련된 모델을 저장하는 함수"""
    import os
    from datetime import datetime

    # 저장 디렉토리 생성
    os.makedirs(save_path, exist_ok=True)

    # 현재 시간을 포함한 파일명 생성
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"{model_name}_{timestamp}.pt"
    model_path = os.path.join(save_path, model_filename)

    # 모델 저장
    model.save(model_path)

    print(f"모델이 저장되었습니다: {model_path}")
    return model_path

save_dir = "/content/models"


## 데이터 불러오기

In [ ]:
# 경로 설정
RAW_DATA_PATH = '/content/drive/MyDrive/(개인)AI_3기/프로젝트/raw_data'
TRAIN_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'train_images')
TRAIN_ANNOTATIONS_PATH = os.path.join(RAW_DATA_PATH, 'train_annotations')
TEST_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'test_images')

print("데이터 경로 확인:")
print(f"Train Images: {TRAIN_IMAGES_PATH}")
print(f"Train Annotations: {TRAIN_ANNOTATIONS_PATH}")
print(f"Test Images: {TEST_IMAGES_PATH}")


In [ ]:
SRC_RAW_DATA="/content/drive/MyDrive/(개인)AI_3기/프로젝트/raw_data"
DST_RAW_DATA="/content/raw_data"

In [ ]:
!mkdir -p "$DST_RAW_DATA"
!rsync -avh --progress "{SRC_RAW_DATA}/" "{DST_RAW_DATA}/"

In [ ]:
# 경로 설정
RAW_DATA_PATH = '/content/raw_data'
TRAIN_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'train_images')
TRAIN_ANNOTATIONS_PATH = os.path.join(RAW_DATA_PATH, 'train_annotations')
TEST_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'test_images')

print("데이터 경로 확인:")
print(f"Train Images: {TRAIN_IMAGES_PATH}")
print(f"Train Annotations: {TRAIN_ANNOTATIONS_PATH}")
print(f"Test Images: {TEST_IMAGES_PATH}")

# COCO 데이터 파싱

In [ ]:
def load_coco_annotations(annotations_path):
    """COCO 형식 어노테이션 파일들을 로드하고 통합"""
    all_annotations = []
    all_images = []
    all_categories = []

    # 모든 JSON 파일 찾기
    json_files = []
    for root, dirs, files in os.walk(annotations_path):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))

    print(f"총 {len(json_files)}개의 어노테이션 파일 발견")

    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 이미지 정보 추가
            if 'images' in data:
                all_images.extend(data['images'])

            # 어노테이션 정보 추가
            if 'annotations' in data:
                all_annotations.extend(data['annotations'])

            # 카테고리 정보 추가
            if 'categories' in data:
                all_categories.extend(data['categories'])

        except Exception as e:
            print(f"Error loading {json_file}: {e}")

    return all_images, all_annotations, all_categories

# 데이터 로드
images, annotations, categories = load_coco_annotations(TRAIN_ANNOTATIONS_PATH)

print(f"총 이미지 수: {len(images)}")
print(f"총 어노테이션 수: {len(annotations)}")
print(f"총 카테고리 수: {len(categories)}")

# 데이터프레임으로 변환
df_images = pd.DataFrame(images)
df_annotations = pd.DataFrame(annotations)
df_categories = pd.DataFrame(categories)

print("\n이미지 데이터 샘플:")
print(df_images.head())
print("\n어노테이션 데이터 샘플:")
print(df_annotations.head())
print("\n카테고리 데이터 샘플:")
print(df_categories.head())

# 데이터 분석

In [ ]:
# 이미지 크기 분석
print("이미지 크기 통계:")
print(df_images[['width', 'height']].describe())

# 이미지 크기 분포 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_images['width'], bins=30, alpha=0.7, color='skyblue')
plt.title('Image Width Distribution')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(df_images['height'], bins=30, alpha=0.7, color='lightcoral')
plt.title('Image Height Distribution')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# 카테고리별 분포
category_counts = df_categories['name'].value_counts()
print(f"\n총 약 종류 수: {len(category_counts)}")

plt.figure(figsize=(15, 8))
category_counts.head(20).plot(kind='bar')
plt.title('Top 20 Drug Categories')
plt.xlabel('Drug Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
category_counts.head(20)

In [ ]:
# 바운딩 박스 크기 분석
bbox_areas = df_annotations['area'].values
bbox_widths = [bbox[2] for bbox in df_annotations['bbox']]
bbox_heights = [bbox[3] for bbox in df_annotations['bbox']]

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(bbox_areas, bins=50, alpha=0.7, color='lightgreen')
plt.title('Bounding Box Area Distribution')
plt.xlabel('Area (pixels²)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(bbox_widths, bins=50, alpha=0.7, color='orange')
plt.title('Bounding Box Width Distribution')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(bbox_heights, bins=50, alpha=0.7, color='purple')
plt.title('Bounding Box Height Distribution')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 데이터 분석
- 이미지 특성
    - 이미지 크기 : 976 x 1280 픽셀 (모두 같음)
    - 총 이미지 수 : 2,332개

- 약 종류 분포
    - 총 약 종류 : 73개
    - Top rank 1 : 가네신에프정(은행엽엑스)(수출용) [514개]
    - Top rank 2 : 일양하이트린정 2mg [240개]
    - Top rank 3 : 보령부스파정 5mg [180개]

    - 클래스 불균형 문제: 상위 약들이 압도적으로 많음

- 바운딩 박스 특성
    - 면적 : 50,000 픽셀^2 이하
    - 너비 : 주로 200~225 픽셀
    - 높이 : 200~250 픽셀, 475~500 픽셀 등


## 이미지 전처리

In [ ]:
def preprocess_image(image_path, target_size=(640, 640)):
    """이미지 전처리 함수"""
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {image_path}")

    # BGR to RGB 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 원본 크기 저장
    original_height, original_width = image.shape[:2]

    # 리사이즈
    resized_image = cv2.resize(image, target_size)

    # 정규화 (0-1 범위)
    normalized_image = resized_image.astype(np.float32) / 255.0

    return {
        'image': normalized_image,
        'original_size': (original_width, original_height),
        'target_size': target_size
    }

def adjust_bbox(bbox, original_size, target_size):
    """바운딩 박스 좌표 조정"""
    x, y, w, h = bbox
    orig_w, orig_h = original_size
    target_w, target_h = target_size

    # 스케일 계산
    scale_x = target_w / orig_w
    scale_y = target_h / orig_h

    # 좌표 변환
    new_x = x * scale_x
    new_y = y * scale_y
    new_w = w * scale_x
    new_h = h * scale_y

    return [new_x, new_y, new_w, new_h]

# 샘플 이미지로 전처리 테스트
sample_image_path = os.path.join(TRAIN_IMAGES_PATH, df_images.iloc[0]['file_name'])
sample_annotation = df_annotations.iloc[0]

try:
    processed = preprocess_image(sample_image_path)
    adjusted_bbox = adjust_bbox(
        sample_annotation['bbox'],
        processed['original_size'],
        processed['target_size']
    )

    print("전처리 결과:")
    print(f"원본 크기: {processed['original_size']}")
    print(f"변환 크기: {processed['target_size']}")
    print(f"원본 바운딩 박스: {sample_annotation['bbox']}")
    print(f"조정된 바운딩 박스: {adjusted_bbox}")

    # 시각화
    plt.figure(figsize=(12, 6))

    # 원본 이미지
    original_img = cv2.imread(sample_image_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    x, y, w, h = sample_annotation['bbox']
    rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title('Original Image with BBox')
    plt.axis('off')

    # 전처리된 이미지
    plt.subplot(1, 2, 2)
    plt.imshow(processed['image'])
    x, y, w, h = adjusted_bbox
    rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title('Preprocessed Image with Adjusted BBox')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"전처리 테스트 중 오류: {e}")

## 데이터 증강

In [14]:
class DrugDetectionTransforms:

    def __init__(self, is_training=True, target_size=(640, 640)):
        self.is_training = is_training
        self.target_size = target_size

        if is_training:
            self.transform = v2.Compose([
                v2.Resize(target_size),
                #v2.RandomHorizontalFlip(p=0.5),
                #v2.RandomVerticalFlip(p=0.3),
                v2.RandomRotation(degrees=90),
                v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                #v2.RandomGrayscale(p=0.1),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = v2.Compose([
                v2.Resize(target_size),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def __call__(self, image, bboxes=None, labels=None):
        # PIL Image로 변환
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)

        # 이미지 변환
        transformed_image = self.transform(image)

        # 바운딩 박스 조정 (간단한 스케일링)
        if bboxes is not None and self.is_training:
            # 랜덤 변환에 따른 바운딩 박스 조정
            adjusted_bboxes = self._adjust_bboxes(bboxes, image.size, self.target_size)
            return transformed_image, adjusted_bboxes, labels

        return transformed_image, bboxes, labels

    def _adjust_bboxes(self, bboxes, original_size, target_size):
        """바운딩 박스 조정 (간단한 스케일링)"""
        orig_w, orig_h = original_size
        target_w, target_h = target_size

        scale_x = target_w / orig_w
        scale_y = target_h / orig_h

        adjusted_bboxes = []
        for bbox in bboxes:
            x, y, w, h = bbox
            new_x = x * scale_x
            new_y = y * scale_y
            new_w = w * scale_x
            new_h = h * scale_y
            adjusted_bboxes.append([new_x, new_y, new_w, new_h])

        return adjusted_bboxes

## 데이터 셋

In [15]:
class DrugDetectionDataset(Dataset):

    def __init__(self, images_df, annotations_df, categories_df,
                 images_path, transform=None, target_size=(640, 640)):
        self.images_df = images_df
        self.annotations_df = annotations_df
        self.categories_df = categories_df
        self.images_path = images_path
        self.transform = transform
        self.target_size = target_size

        # 카테고리 ID 매핑 생성
        self.category_to_id = {cat['name']: cat['id'] for _, cat in categories_df.iterrows()}
        self.id_to_category = {cat['id']: cat['name'] for _, cat in categories_df.iterrows()}

        # 이미지 ID별 어노테이션 그룹화
        self.image_annotations = {}
        for _, ann in annotations_df.iterrows():
            img_id = ann['image_id']
            if img_id not in self.image_annotations:
                self.image_annotations[img_id] = []
            self.image_annotations[img_id].append(ann)

    def __len__(self):
        return len(self.images_df)

    def __getitem__(self, idx):
        # 이미지 정보 가져오기
        img_info = self.images_df.iloc[idx]
        img_path = os.path.join(self.images_path, img_info['file_name'])

        # 이미지 로드 (PIL 사용)
        image = Image.open(img_path).convert('RGB')

        # 어노테이션 가져오기
        img_id = img_info['id']
        annotations = self.image_annotations.get(img_id, [])

        # 바운딩 박스와 라벨 준비
        bboxes = []
        labels = []

        for ann in annotations:
            bbox = ann['bbox']  # [x, y, w, h]
            category_id = ann['category_id']
            bboxes.append(bbox)
            labels.append(category_id)

        # 데이터 증강 적용
        if self.transform:
            image, bboxes, labels = self.transform(image, bboxes, labels)

        return {
            'image': image,
            'bboxes': torch.tensor(bboxes, dtype=torch.float32) if bboxes else torch.empty((0, 4)),
            'labels': torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            'image_id': img_id,
            'original_size': (img_info['width'], img_info['height'])
        }

In [ ]:
# 데이터 증강 파이프라인 생성
train_transform = DrugDetectionTransforms(is_training=True, target_size=(640, 640))
val_transform = DrugDetectionTransforms(is_training=False, target_size=(640, 640))

# 데이터셋 생성
train_dataset = DrugDetectionDataset(
    df_images, df_annotations, df_categories,
    TRAIN_IMAGES_PATH,
    transform=train_transform
)

# 데이터로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

print(f"데이터셋 크기: {len(train_dataset)}")
print(f"데이터로더 배치 수: {len(train_loader)}")

-----

# 모델

## YOLO
 - https://docs.ultralytics.com/models/

### YOLO v8

#### YOLO v8 설정

In [ ]:
!pip install ultralytics

In [ ]:
# YOLO v5 설정 및 확인
from ultralytics import YOLO
import os
import shutil

In [ ]:
# YOLO v5 설정 및 확인
from ultralytics import YOLO
import os
import shutil

# YOLO v8 모델 로드 테스트
model = YOLO('yolov8x.pt')  # 작은 모델로 시작
print("YOLO v8 모델 로드 성공!")

# 현재 디렉토리 구조 확인
print(f"현재 작업 디렉토리: {os.getcwd()}")

#### YOLO v8 데이터셋 변환

In [19]:
def convert_to_yolo_format(images_df, annotations_df, categories_df,
                          images_path, output_path):
    """COCO 형식을 YOLO 형식으로 변환"""

    # 출력 디렉토리 생성
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(os.path.join(output_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'labels'), exist_ok=True)

    # 고유한 카테고리만 추출하고 0부터 시작하는 인덱스 매핑
    unique_categories = categories_df['name'].unique()
    category_mapping = {}

    # 원본 카테고리 ID를 0부터 시작하는 인덱스로 매핑
    for idx, cat_name in enumerate(unique_categories):
        # 해당 이름을 가진 카테고리의 원본 ID 찾기
        original_id = categories_df[categories_df['name'] == cat_name]['id'].iloc[0]
        category_mapping[original_id] = idx

    print(f"총 {len(category_mapping)}개 카테고리 매핑 완료")
    print(f"매핑 예시: {list(category_mapping.items())[:5]}")

    # 이미지 ID별 어노테이션 그룹화
    image_annotations = {}
    for _, ann in annotations_df.iterrows():
        img_id = ann['image_id']
        if img_id not in image_annotations:
            image_annotations[img_id] = []
        image_annotations[img_id].append(ann)

    # 각 이미지에 대해 YOLO 형식으로 변환
    for idx, (_, img_info) in enumerate(images_df.iterrows()):
        img_id = img_info['id']
        img_filename = img_info['file_name']

        # 이미지 복사
        src_img_path = os.path.join(images_path, img_filename)
        dst_img_path = os.path.join(output_path, 'images', img_filename)
        shutil.copy2(src_img_path, dst_img_path)

        # 라벨 파일 생성
        label_filename = img_filename.replace('.png', '.txt')
        label_path = os.path.join(output_path, 'labels', label_filename)

        annotations = image_annotations.get(img_id, [])

        with open(label_path, 'w') as f:
            for ann in annotations:
                # 바운딩 박스 좌표 (COCO: [x, y, w, h] -> YOLO: [x_center, y_center, w, h])
                bbox = ann['bbox']
                x, y, w, h = bbox

                # 이미지 크기
                img_width = img_info['width']
                img_height = img_info['height']

                # YOLO 형식으로 변환 (정규화된 좌표)
                x_center = (x + w/2) / img_width
                y_center = (y + h/2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height

                # 카테고리 ID (YOLO는 0부터 시작)
                category_id = category_mapping[ann['category_id']]

                # YOLO 형식: class_id x_center y_center width height
                f.write(f"{category_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        if (idx + 1) % 100 == 0:
            print(f"진행률: {idx + 1}/{len(images_df)}")

    # classes.txt 파일 생성
    classes_path = os.path.join(output_path, 'classes.txt')
    with open(classes_path, 'w', encoding='utf-8') as f:
        for _, cat in categories_df.iterrows():
            f.write(f"{cat['name']}\n")

    print(f"YOLO 형식 변환 완료! 출력 경로: {output_path}")
    return output_path

In [ ]:
# 데이터셋 분할 (훈련/검증)
from sklearn.model_selection import train_test_split

if os.path.exists('/content/raw_data/data_lyh/yolo_dataset'):
    shutil.rmtree('/content/raw_data/data_lyh/yolo_dataset')

train_images, val_images = train_test_split(
    df_images, test_size=0.2, random_state=42
)

print(f"훈련 데이터: {len(train_images)}개")
print(f"검증 데이터: {len(val_images)}개")

# YOLO 형식으로 변환
yolo_dataset_path = '/content/raw_data/data_lyh/yolo_dataset'

# 훈련 데이터 변환
train_path = os.path.join(yolo_dataset_path, 'train')
convert_to_yolo_format(
    train_images, df_annotations, df_categories,
    TRAIN_IMAGES_PATH, train_path
)

# 검증 데이터 변환
val_path = os.path.join(yolo_dataset_path, 'val')
convert_to_yolo_format(
    val_images, df_annotations, df_categories,
    TRAIN_IMAGES_PATH, val_path
)

print("데이터셋 변환 완료!")

In [ ]:
# 카테고리 수 변화 원인 확인
print("이전 고유 카테고리 수:", len(df_categories['name'].unique()))

# 중복 제거 확인
unique_categories = df_categories['name'].unique()
print("실제 고유 카테고리:", len(unique_categories))

In [ ]:
# 라벨 파일 내용 확인
import os
label_files = os.listdir('/content/raw_data/data_lyh/yolo_dataset/train/labels')
sample_label_path = os.path.join('/content/raw_data/data_lyh/yolo_dataset/train/labels', label_files[0])

with open(sample_label_path, 'r') as f:
    print("샘플 라벨 파일 내용:")
    print(f.read())

In [23]:
# YAML 파일 내용 확인
"""
with open('/content/raw_data/data_lyh/yolo_dataset/dataset.yaml', 'r') as f:
    print("YAML 파일 내용:")
    print(f.read())
"""

'\nwith open(\'/content/raw_data/data_lyh/yolo_dataset/dataset.yaml\', \'r\') as f:\n    print("YAML 파일 내용:")\n    print(f.read())\n'

In [ ]:
# 현재 매핑 상태 확인
print("원본 카테고리 ID들:")
print(df_categories['id'].unique()[:10])  # 처음 10개 확인

#print("\n매핑된 카테고리 ID들:")
#print(list(category_mapping.items())[:10])  # 처음 10개 확인

#### YOLO 설정 파일 생성

In [ ]:
# 카테고리별 분포
category_counts = df_categories['name'].value_counts()
print(f"\n총 약 종류 수: {len(category_counts)}")

In [ ]:
# 중복된 카테고리 제거하고 고유한 약 이름만 추출
unique_categories = df_categories['name'].unique()
unique_categories_list = unique_categories.tolist()

print(len(unique_categories_list))
print(unique_categories_list)

In [ ]:
# YOLO 설정 파일 생성
yaml_content = f"""
# YOLO v8 설정 파일
path: {os.path.abspath(yolo_dataset_path)}  # 데이터셋 루트 경로
train: train/images  # 훈련 이미지 경로
val: val/images      # 검증 이미지 경로

# 클래스 수 (고유한 약 종류 수)
nc: {len(unique_categories)}  # 클래스 개수

# 클래스 이름
names: {unique_categories_list}
"""

# 설정 파일 저장
yaml_path = os.path.join(yolo_dataset_path, 'dataset.yaml')
with open(yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"YOLO 설정 파일 생성 완료: {yaml_path}")
print(f"총 클래스 수: {len(unique_categories)}")
print(f"고유한 약 종류: {len(unique_categories)}개")

# YAML 파일 내용 확인
print("\nYAML 파일 내용:")
with open(yaml_path, 'r', encoding='utf-8') as f:
    print(f.read())

In [ ]:
# 라벨 파일 내용 확인
label_path = '/content/raw_data/data_lyh/yolo_dataset/train/labels/K-001900-010224-016551-031705_0_2_0_2_70_000_200.txt'
with open(label_path, 'r') as f:
    print(f.read())

In [ ]:
# YAML 파일 내용 확인
with open('/content/raw_data/data_lyh/yolo_dataset/dataset.yaml', 'r') as f:
    print(f.read())

#### YOLO 모델 학습

In [31]:
!wget -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8x.pt \
      -O /content/yolov8x.pt

In [ ]:
import torch
print(torch.__version__, torch.version.cuda)

2.7.1+cu118 11.8

In [ ]:
# YOLO v8 모델 학습
def train_yolo_v8_model():
    """YOLO v8 모델 학습"""

    # 모델 초기화 (v8로 변경)
    model = YOLO('/content/yolov8n.pt')  # nano 버전부터 시작

    # 학습 설정 (v8 최적화)
    results = model.train(
        data=yaml_path,           # 기존 dataset.yaml 그대로 사용
        epochs=100,               # 에포크 수
        imgsz=640,               # 이미지 크기 (640, 512, 416)
        batch=8,                 # 배치 크기 (8, 4, 2)
        device=device,           # 디바이스 (MPS/CPU)
        patience=10,             # Early stopping patience
        save=True,               # 모델 저장
        # save_period=1,            # 매 에폭마다 저장
        project='drug_detection_v8n_b8_imgv3', # 프로젝트 이름
        name='yolov8n_drug_model_b8_imgv3', # 실험 이름
        exist_ok=True,           # 기존 실험 덮어쓰기
        pretrained=True,         # 사전 훈련된 가중치 사용
        optimizer='Adam',        # 옵티마이저
        lr0=0.001,              # 초기 학습률
        weight_decay=0.0005,     # 가중치 감쇠
        warmup_epochs=3,         # 워밍업 에포크
        cos_lr=True,            # 코사인 학습률 스케줄링
        half=True,            # 메모리 절약용
        format='pt',

        #augment=True,                 # 증강 기능 켜기
        #hsv_h=0.02,                  # 색조(hue) 변동 폭: ±2%
        #hsv_s=0.4,                    # 채도(saturation) 변동 폭: ±40%
        #hsv_v=0.6,                    # 명도(value) 변동 폭: ±40%
        #degrees=20.0,                 # 회전 각도 범위: ±20°
        #translate=0.2,                # 평행 이동: ±20% (x/y)
        #scale=0.5,                    # 크기 조정: 0.5×–1.5×
        #shear=2.0,                    # 전단(shear) 각도: ±2°

    )

    return results, model

# 학습 시작
print("YOLO 모델 학습 시작...")
results, trained_model = train_yolo_v8_model()
print("학습 완료!")

saved_model_path = save_model(trained_model, save_dir)

# 학습 결과 시각화
#results.plot()  # 학습 곡선 플롯

In [ ]:
def train_yolo_v11n_model(yaml_path: str, save_dir: str, device: str = 'cpu'):
    """YOLO v11 nano 모델 학습"""

    # 1) 모델 초기화 (v11 nano)
    model = YOLO('/content/yolo11n.pt')

    # 2) 학습 설정
    results = model.train(
        data=yaml_path,            # dataset.yaml 경로
        epochs=100,                # 총 에포크 수
        imgsz=640,                 # 입력 이미지 크기
        batch=8,                   # 배치 크기
        device=device,             # 'cpu', 'cuda:0' 등
        patience=10,               # EarlyStopping patience
        save=True,                 # 최종 모델 저장
        project='drug_detection_v11n',  # 저장할 프로젝트 폴더명
        name='yolov11_drug_model',      # 실험 이름
        exist_ok=True,             # 동일 이름 실험 덮어쓰기
        pretrained=True,           # 사전 훈련된 가중치 사용
        optimizer='Adam',          # 옵티마이저
        lr0=0.001,                 # 초기 학습률
        weight_decay=0.0005,       # 가중치 감쇠
        warmup_epochs=3,           # 워밍업 에포크 수
        cos_lr=True,               # 코사인 러닝레이트 스케줄러
        amp=True,                  # 자동 Mixed Precision (v8의 half=True 대체)
        time=None,                 # 학습 시간 제한(시간 단위), None이면 제한 없음
        close_mosaic=10,           # 마지막 10 에포크에서 mosaic 강화 끄기
        fraction=1.0,              # 학습 데이터 비율 (0~1)
        profile=False,             # ONNX/TensorRT 속도 프로파일링
        freeze=[0],                # 고정할 레이어 번호 리스트
        format='pt'                # 저장 포맷
    )

    return results, model

### 모델 불러오기

In [30]:
saved_model_path = '/content/best_19.pt'

In [50]:
saved_model_path = '/content/drug_detection_v8n_b8_imgv3/yolov8n_drug_model_b8_imgv3/weights/best.pt'

In [46]:
saved_model_path = '/content/drug_detection_v8n_b32_imgv3/yolov8n_drug_model_b32_imgv3/weights/best.pt'

In [42]:
saved_model_path = '/content/drug_detection_v8x_b32_imgv3/yolov8x_drug_model_b32_imgv3/weights/best.pt'

In [38]:
saved_model_path = '/content/drug_detection_v8x_b32_imgv3_aug2/yolov8x_drug_model_b32_imgv3_aug2/weights/best.pt'

In [34]:
saved_model_path = '/content/drug_detection_v8x_b32_imgv3_aug/yolov8x_drug_model_b32_imgv3_aug/weights/best.pt'

In [52]:
saved_model_path = '/content/drug_detection_v8x_b16_imgv3/yolov8x_drug_model_b16_imgv3/weights/best.pt'

In [47]:
saved_model_path = '/content/drug_detection_v8x_b32_imgv3/yolov8x_drug_model_b32_imgv3/weights/best.pt'

In [40]:
saved_model_path = '/content/drug_detection_v8l_b8_imgv3/yolov8l_drug_model_b8_imgv3/weights/best.pt'

In [34]:
saved_model_path = '/content/drug_detection_v8n_b8_imgv3/yolov8n_drug_model_b8_imgv3/weights/best.pt'

In [ ]:
saved_model_path = '/content/drug_detection_v8x_b32_imgv3/yolov8x_drug_model_b32_imgv3/weights/best.pt'

In [ ]:
saved_model_path = '/content/drug_detection_v8x_b8_imgv3/yolov8x_drug_model_b8_imgv3/weights/best.pt'

In [ ]:
saved_model_path = '/content/drug_detection_model_YOLOv5x_b8_e100(61)_20250723_1.pt'

In [ ]:
saved_model_path = '/content/drug_detection_model_YOLOv5x_b32_e100(85)_20250723_2.pt'

In [ ]:
saved_model_path = '/content/models/drug_detection_model_20250723_044917.pt'

In [ ]:
saved_model_path = '/content/drug_detection_v8x_b32/yolov8x_drug_model_b32/weights/best.pt'

In [ ]:
saved_model_path = '/content/drug_detection_v8s/yolov8_drug_model/weights/best.pt'

In [ ]:
# 모델 불러오기 함수
def load_model(model_path):
    try:
        # 모델 로드
        model = YOLO(model_path)
        print(f"모델이 성공적으로 로드되었습니다: {model_path}")
        return model
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        return None

# 모델 불러오기 실행
loaded_model = load_model(saved_model_path)

### 모델 성능 평가

In [ ]:
# 모델 성능 평가 함수
def evaluate_model(model, data_path):

    # 검증 데이터셋에서 평가
    results = model.val(data=data_path)

    # 결과 출력
    print("=== 모델 성능 평가 결과 ===")
    print(f"mAP50: {results.box.map50:.3f}")
    print(f"mAP50-95: {results.box.map:.3f}")
    print(f"Precision: {results.box.mp:.3f}")
    print(f"Recall: {results.box.mr:.3f}")

    return results

# 모델 평가 실행
if loaded_model:
    evaluation_results = evaluate_model(loaded_model, yaml_path)

In [ ]:
from ultralytics import YOLO

# 1. Load your trained .pt weights
model = YOLO(saved_model_path)

# 2. Run inference (detect) on a directory of images
#    – source can be a folder, a single image, or even a video file
results = model.predict(
    source='/content/raw_data/data_lyh/yolo_dataset/val/images/',
    imgsz=640,        # image resize
    conf=0.25,        # confidence threshold
    save=True         # save annotated images to runs/detect/
)

# 3. Inspect the results
for r in results:
    # r.boxes.xyxy, r.boxes.conf, r.boxes.cls
    print(r)

In [ ]:
# 모델 성능 평가 함수
def evaluate_model(model, data_path):

    # 검증 데이터셋에서 평가
    results = model.val(data=data_path)

    # 결과 출력
    print("=== 모델 성능 평가 결과 ===")
    print(f"mAP50: {results.box.map50:.3f}")
    print(f"mAP50-95: {results.box.map:.3f}")
    print(f"Precision: {results.box.mp:.3f}")
    print(f"Recall: {results.box.mr:.3f}")

    return results

# 모델 평가 실행
if loaded_model:
    evaluation_results = evaluate_model(model, '/content/dataset.yaml')

### 모델 테스트

In [ ]:
# 모델 추론 테스트 함수
def test_inference(model, image_path):

    # 이미지 로드
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 추론 실행
    results = model(image_path)

    # 결과 시각화
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            # 바운딩 박스 그리기
            for box in boxes:
                # 좌표 추출
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                cls = int(box.cls[0].cpu().numpy())

                # 박스 그리기
                cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

                # 라벨 추가
                label = f"Class {cls}: {conf:.2f}"
                cv2.putText(image_rgb, label, (int(x1), int(y1)-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 결과 이미지 표시
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    plt.title("Drug Detection Results")
    plt.axis('off')
    plt.show()

    return results

In [ ]:
# 테스트 실행 (샘플 이미지 경로 지정 필요)
sample_image_path = "/content/raw_data/data_lyh/yolo_dataset/val/images/K-001900-010224-016551-031705_0_2_0_2_90_000_200.png"
if loaded_model and os.path.exists(sample_image_path):
    inference_results = test_inference(loaded_model, sample_image_path)
else:
    if os.path.exists(sample_image_path):
        print("a")
    else:
        print("b")

In [ ]:
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import subprocess

# Colab/노트북에 한글폰트 설치 (한번만 실행)
subprocess.run(['apt-get', 'update', '-qq'])
subprocess.run(['apt-get', 'install', '-qq', 'fonts-nanum'])

# 2) 폰트 로드 (NanumGothic 예시)
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font = ImageFont.truetype(font_path, size=20)

In [ ]:
import os
from subprocess import run

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

if not os.path.exists(font_path):
    run(['sudo','apt-get','install','-y','fonts-nanum'], check=True)
    run(['sudo','fc-cache','-fv'], check=True)
    # matplotlib 캐시 지우기
    run(['rm','-rf', os.path.expanduser('~/.cache/matplotlib')], check=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings

warnings.filterwarnings(action='ignore')
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'           # 나눔 고딕
fm.fontManager.addfont(path)
plt.rc('font', family=fm.FontProperties(fname=path, size=10).get_name()) # 기본 폰트 사이즈 : 10

In [ ]:
# 1) YAML에서 클래스 이름 한 번만 로드
with open('/content/raw_data/data_lyh/yolo_dataset/dataset.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
class_names = data_cfg['names']  # 리스트: 인덱스 0~72 → 약 이름

In [ ]:
# 2) 추론 및 시각화 함수
def test_inference2(model, image_path, class_names=class_names):
    """
    YOLO 모델로 이미지 추론 후, 박스 위에 약 이름과 confidence를 그려서 보여줍니다.
    """
    # 이미지 로드 & RGB 변환
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 추론 실행 (results: list of Results)
    results = model(image_path)

    pil_img = Image.fromarray(image_rgb)
    draw   = ImageDraw.Draw(pil_img)

    # 결과 시각화
    for res in results:
        for box in res.boxes:
            x1,y1,x2,y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf       = float(box.conf[0].cpu().numpy())
            cls        = int(box.cls[0].cpu().numpy())
            name       = class_names[cls]
            label      = f"{name} {conf:.2f}"

            # 박스
            draw.rectangle([x1,y1,x2,y2], outline="red", width=2)

            # 한글 텍스트
            draw.text((x1, y1-20), label, font=font, fill="red")

    # 5) 다시 NumPy → Matplotlib 출력
    out = np.array(pil_img)
    plt.figure(figsize=(10,6))
    plt.imshow(out)
    plt.axis('off')
    plt.show()
    return results


    return results




In [ ]:
sample_image_path = "/content/raw_data/data_lyh/yolo_dataset/val/images/K-001900-010224-016551-031705_0_2_0_2_70_000_200.png"
if loaded_model and os.path.exists(sample_image_path):
    inference_results = test_inference2(loaded_model, sample_image_path)

In [ ]:
import glob

# 1) 모델 로드
model = YOLO(loaded_model)

# 2) 테스트 이미지 경로 리스트 (경로는 실제 위치에 맞게 바꿔주세요)
test_dir = '/content/raw_data/test_images'
image_paths = sorted(glob.glob(os.path.join(test_dir, '*.png')))

# 3) 결과를 담아둘 리스트
rows = []

# 4) 각 이미지에 대해 추론 → SubmissionString 생성
for img_path in image_paths:
    img_id = os.path.basename(img_path)
    # conf threshold, device 등은 필요에 맞게 조절
    results = model.predict(source=img_path, conf=0.25, device=0)

    preds = []
    for res in results:            # 리스트지만 보통 1개
        for box in res.boxes:      # res.boxes: DetectMultiBackend output
            cls   = int(box.cls[0].cpu().numpy())
            conf  = float(box.conf[0].cpu().numpy())
            x1,y1,x2,y2 = box.xyxy[0].cpu().numpy()
            # 예: class conf x1 y1 x2 y2 (절대 좌표)
            preds.append(f"{cls} {conf:.2f} {x1:.1f} {y1:.1f} {x2:.1f} {y2:.1f}")

    # 예: PredictionString 열에 공백으로 구분된 예측 문자열을 합칩니다
    pred_str = " ".join(preds)
    rows.append({
        'image_id': img_id,
        'PredictionString': pred_str
    })

# 5) DataFrame 생성 및 CSV 저장
submission = pd.DataFrame(rows, columns=['image_id','PredictionString'])
submission.to_csv('submission.csv', index=False)

print("submission.csv 생성 완료:", submission.head())